In [1]:
import os
import sys
from langchain_community.vectorstores import Chroma
from IPython.display import display, Markdown

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from src.routing import Router
from enterprise_knowledge_retriever.src.document_retrieval import DocumentRetrieval

CONFIG_PATH = os.path.join(kit_dir,'config.yaml')

/Users/feiwu/Documents/ai-starter-kit-personal/ai-starter-kit/routing/routing_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/feiwu/Documents/ai-starter-kit-personal/ai-starter-kit/routing/routing_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/feiwu/Documents/ai-starter-kit-personal/ai-starter-kit/routing/routing_env/lib/python3.9/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


WANDB_API_KEY is not set. Weave initialization skipped.


# init router

In [2]:
router = Router(CONFIG_PATH)

# init RAG

In [3]:
document_retrieval =  DocumentRetrieval()
embeddings = document_retrieval.load_embedding_model()
# print(os.path.join(kit_dir, "data/800_20"))
vectorstore = document_retrieval.load_vdb(db_path = os.path.join(kit_dir, "data/800_20"), embeddings=embeddings)
document_retrieval.init_retriever(vectorstore)
conversation = document_retrieval.get_qa_retrieval_chain()

2024-09-06 14:30:40,557 [INFO] - Load pretrained SentenceTransformer: intfloat/e5-large-v2


load INSTRUCTOR_Transformer


/Users/feiwu/Documents/ai-starter-kit-personal/ai-starter-kit/routing/routing_env/lib/python3.9/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
2024-09-06 14:30:42,700 [INFO] - Use pytorch device: cpu


max_seq_length  512
Loading collection name is None


2024-09-06 14:30:42,934 [INFO] - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


# user input

In [4]:
# query = "How to measure performance using sambatune?"
# query = "Who is the CEO of Sambanova?"
query = "What is LLM?"

# pipeline

In [5]:
datasource = router.routing(query)
if datasource == "vectorstore":
    output = conversation.invoke({"question":query})
    response = output['answer']
else:
    response = document_retrieval.llm(query)
print(response)

/Users/feiwu/Documents/ai-starter-kit-personal/ai-starter-kit/routing/routing_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


LLM can refer to several things, but I'll cover the most common meanings:

1. **Large Language Model**: In natural language processing (NLP), a Large Language Model (LLM) is a type of artificial intelligence (AI) model that is trained on a massive corpus of text data to generate human-like language. These models are designed to understand and generate text, and they can be used for a wide range of applications, such as language translation, text summarization, and chatbots.
2. **Law Library of Congress**: The Law Library of Congress (LLM) is a research library that provides access to legal information and resources for the U.S. Congress and the public. It is part of the Library of Congress and is one of the largest and most comprehensive law libraries in the world.
3. **Language Learning Method**: In language education, LLM can refer to a Language Learning Method, which is a systematic approach to learning a language. This can include various techniques, such as immersion, language exc

# response router

In [6]:
router.init_response_router()
reroute = router.check_response(query, response, datasource)
if reroute == "true" and datasource == "vectorstore":
    print("Rerouting to LLM")
    # send the query to the llm instead
    revised_answer = document_retrieval.llm(query)
    # printing the new answer
    print("Answer: " + "\n" + revised_answer)

elif reroute == "true" and datasource == "llm":
    print("Rerouting to Vectorstore")
    # send the query to the vectorstore instead
    revised_answer = conversation.invoke({"question":query})
    print("Answer: " + "\n" + revised_answer["answer"])

Rerouting to Vectorstore


TypeError: can only concatenate str (not "dict") to str

In [7]:
revised_answer

{'question': 'What is LLM?',
 'answer': 'According to the provided context, Large Language Models (LLMs) are a type of foundation model specifically focused on language, which enables them to generate an understanding of the context of language and generate new language content. They are an emerging type of transformer model that can deliver a high degree of accuracy across dozens of different tasks, including text generation, classification, question answering, sentiment analysis, and entailment.',
 'source_documents': [Document(metadata={'source': 'blog_enabling-open-source-llms-to-become-effective-tool-manipulators.txt'}, page_content='# Enabling Open Source LLMs to Become Effective Tool Manipulators\n\n#### **Introduction**'),
  Document(metadata={'source': 'blog_achieving-gpt-175b-level-accuracy-with-a-10x-more-efficient-model.txt'}, page_content='To explore new tasks, in-context learning is a powerful technique. It allows\nrapid prototyping to determine whether LLMs can be deploy